In [1]:
import xgboost as xgb
import pandas as pd
pd.set_option('display.max_columns', None)
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score, recall_score
import numpy as np
from tqdm import tqdm_notebook

#每次可以输出多个变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [99]:
usecols = ['register_type', 'device_type', 'user_reg_days', 'user_lastweek_launchday', 'user_last_launch_dist', 'user_hist_launch_freq', 'user_hist_launchday', 'user_mean_continue_launch_times_lastweek', 'user_max_continue_launch_times_lastweek', 'user_activity_days_hist', 'user_activity_days_lastweek', 'user_min_continue_launch_times_lastweek', 'user_mean_continue_launch_days_lastweek', 'user_max_continue_launch_days_lastweek', 'user_mean_continue_activity_days_lastweek', 'user_max_continue_activity_days_lastweek', 'user_lastweek_act_0_freq', 'user_lastweek_actcount', 'user_lastweek_act_video_uniquecount', 'user_max_continue_launch_times_hist', 'user_min_continue_launch_days_lastweek', 'user_mean_continue_launch_times_hist', 'user_min_continue_activity_days_lastweek', 'user_hist_act_0_count', 'user_hist_actcount', 'user_mean_continue_activity_days_hist', 'user_max_continue_launch_days_hist', 'user_hist_act_freq', 'user_mean_continue_launch_days_hist', 'user_hist_act_author_count', 'user_mean_no_launch_days_hist', 'user_min_activity_daytimes_lastweek', 'user_lastweek_act', 'user_lastweek_act_0', 'user_mean_continue_activity_times_hist', 'user_max_launch_daytimes_lastweek', 'user_lastweek_launch', 'user_lastweek_act_page_3_count', 'user_lastweek_act_page_1_count', 'user_max_no_launch_days_hist', 'user_last_act_date', 'user_lastweek_act_2_freq', 'user_lastweek_video_freq', 'user_lastweek_act_2_count', 'user_lastweek_launch_freq', 'user_max_no_activity_days_lastweek_hist_dist', 'user_lastweek_act_page_2_count', 'user_var_continue_activity_times_lastweek', 'user_kurt_continue_activity_days_hist', 'user_launch_range_percent', 'user_activity_div_launch_days_hist', 'user_hist_act_video_meancount', 'user_hist_video_activity_types', 'user_activity_range_percent', 'user_5daybefore_act_page_1_count', 'user_min_continue_activity_times_5daywin', 'user_lastweek_hist_act_3_count_dist', 'user_lastweek_hist_act_page_3_count_dist', 'user_mean_createvideo_date_lastweek_hist_dist', 'user_4daybefore_act_page_4_count', 'user_kurt_no_launch_days_6daywin', 'user_max_continue_createvideo_days_4daywin', 'user_2daybefore_act_1_count', 'user_max_continue_createvideo_days_hist']
by = ['user_id', 'data_weeknum', 'data_weekstart']
target = ['label']
len(usecols)

64

In [100]:
df = pd.read_csv('./features/b/baseline_features12_ab.csv', usecols=usecols+by+target)
df.shape

/usr/local/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(198057, 68)

In [4]:
cols = ["register_type","device_type","user_reg_days","user_lastweek_launchday","user_last_launch_dist","user_max_continue_launch_times_lastweek","user_activity_days_hist","user_activity_days_lastweek","user_mean_continue_launch_days_lastweek","user_max_continue_launch_days_lastweek","user_mean_continue_activity_days_lastweek","user_max_continue_activity_days_lastweek","user_lastweek_actcount","user_lastweek_act_video_uniquecount","user_max_continue_launch_times_hist","user_min_continue_launch_days_lastweek","user_mean_continue_launch_times_hist","user_min_continue_activity_days_lastweek","user_hist_actcount","user_mean_continue_activity_days_hist","user_max_continue_launch_days_hist","user_hist_act_freq","user_mean_continue_launch_days_hist","user_hist_act_author_count","user_mean_no_launch_days_hist","user_min_activity_daytimes_lastweek","user_lastweek_act","user_lastweek_act_0","user_mean_continue_activity_times_hist","user_max_launch_daytimes_lastweek","user_lastweek_launch","user_lastweek_act_page_3_count","user_lastweek_act_page_1_count","user_max_no_launch_days_hist","user_last_act_date","user_lastweek_act_2_freq","user_lastweek_video_freq","user_lastweek_act_2_count","user_lastweek_launch_freq","user_max_no_activity_days_lastweek_hist_dist","user_lastweek_act_page_2_count","user_var_continue_activity_times_lastweek","user_kurt_continue_activity_days_hist","user_launch_range_percent","user_activity_div_launch_days_hist","user_activity_range_percent","user_5daybefore_act_page_1_count","user_min_continue_activity_times_5daywin","user_lastweek_hist_act_3_count_dist","user_4daybefore_act_page_4_count","user_kurt_no_launch_days_6daywin","user_max_continue_createvideo_days_4daywin","user_2daybefore_act_1_count","user_max_continue_createvideo_days_hist","(device_type-user_activity_days_lastweek)","(user_lastweek_launchday-user_min_continue_launch_days_lastweek)","(user_last_launch_dist/user_lastweek_act_video_uniquecount)","(user_activity_days_hist-user_kurt_continue_activity_days_hist)","(user_activity_days_hist/user_var_continue_activity_times_lastweek)","(user_activity_days_lastweek-user_max_continue_launch_times_hist)","(user_activity_days_lastweek/user_max_continue_activity_days_lastweek)","(user_lastweek_actcount-user_activity_days_lastweek)","(user_lastweek_act_video_uniquecount-user_max_continue_activity_days_lastweek)","(user_min_continue_launch_days_lastweek*user_2daybefore_act_1_count)","(user_hist_act_author_count*user_max_continue_createvideo_days_4daywin)","(user_max_launch_daytimes_lastweek*user_5daybefore_act_page_1_count)","(user_max_launch_daytimes_lastweek*user_lastweek_act_page_3_count)"]
len(cols)

67

In [5]:
cols2 = ['register_type','device_type','user_reg_days','user_last_launch_dist','user_hist_launch_freq','user_hist_launchday','user_max_continue_launch_times_lastweek','user_activity_days_hist','user_activity_days_lastweek','user_min_continue_launch_times_lastweek','user_mean_continue_launch_days_lastweek','user_max_continue_launch_days_lastweek','user_mean_continue_activity_days_lastweek','user_max_continue_activity_days_lastweek','user_lastweek_act_0_freq','user_lastweek_actcount','user_lastweek_act_video_uniquecount','user_min_continue_launch_days_lastweek','user_mean_continue_launch_times_hist','user_min_continue_activity_days_lastweek','user_hist_actcount','user_mean_continue_activity_days_hist','user_max_continue_launch_days_hist','user_hist_act_freq','user_mean_continue_launch_days_hist','user_hist_act_author_count','user_mean_no_launch_days_hist','user_min_activity_daytimes_lastweek','user_lastweek_act','user_lastweek_act_0','user_mean_continue_activity_times_hist','user_max_launch_daytimes_lastweek','user_lastweek_launch','user_lastweek_act_page_3_count','user_lastweek_act_page_1_count','user_max_no_launch_days_hist','user_last_act_date','user_lastweek_act_2_freq','user_lastweek_video_freq','user_lastweek_act_2_count','user_lastweek_launch_freq','user_max_no_activity_days_lastweek_hist_dist','user_lastweek_act_page_2_count','user_var_continue_activity_times_lastweek','user_kurt_continue_activity_days_hist','user_launch_range_percent','user_activity_div_launch_days_hist','user_hist_act_video_meancount','user_hist_video_activity_types','user_activity_range_percent','user_5daybefore_act_page_1_count','user_min_continue_activity_times_5daywin','user_lastweek_hist_act_3_count_dist','user_lastweek_hist_act_page_3_count_dist','user_mean_createvideo_date_lastweek_hist_dist','user_4daybefore_act_page_4_count','user_kurt_no_launch_days_6daywin','user_max_continue_createvideo_days_4daywin','user_2daybefore_act_1_count','user_max_continue_createvideo_days_hist','(register_type+user_lastweek_video_freq)','(register_type*user_hist_launchday)','(user_hist_launchday/user_2daybefore_act_1_count)','(user_activity_days_lastweek-user_mean_continue_activity_days_lastweek)','(user_min_continue_launch_times_lastweek*user_lastweek_hist_act_3_count_dist)','(user_mean_continue_activity_days_lastweek+user_max_no_activity_days_lastweek_hist_dist)','(user_lastweek_act_0_freq-user_4daybefore_act_page_4_count)','(user_max_continue_launch_days_hist*user_5daybefore_act_page_1_count)','(user_var_continue_activity_times_lastweek*user_5daybefore_act_page_1_count)','(user_launch_range_percent/user_4daybefore_act_page_4_count)','(user_launch_range_percent*user_4daybefore_act_page_4_count)','(user_5daybefore_act_page_1_count/user_lastweek_launch)','(user_5daybefore_act_page_1_count*user_min_continue_launch_days_lastweek)','(user_4daybefore_act_page_4_count/user_max_launch_daytimes_lastweek)']
len(cols2)

74

In [101]:
cols = ['register_type', 'device_type', 'user_reg_days', 'user_lastweek_launchday', 'user_last_launch_dist', 'user_hist_launchday', 'user_max_continue_launch_times_lastweek', 'user_activity_days_lastweek', 'user_min_continue_launch_times_lastweek', 'user_mean_continue_launch_days_lastweek', 'user_max_continue_launch_days_lastweek', 'user_lastweek_actcount', 'user_lastweek_act_video_uniquecount', 'user_max_continue_launch_times_hist', 'user_min_continue_launch_days_lastweek', 'user_hist_act_0_count', 'user_hist_actcount', 'user_mean_continue_activity_days_hist', 'user_max_continue_launch_days_hist', 'user_hist_act_freq', 'user_mean_continue_launch_days_hist', 'user_hist_act_author_count', 'user_mean_no_launch_days_hist', 'user_min_activity_daytimes_lastweek', 'user_lastweek_act', 'user_lastweek_act_0', 'user_mean_continue_activity_times_hist', 'user_max_launch_daytimes_lastweek', 'user_lastweek_launch', 'user_lastweek_act_page_3_count', 'user_lastweek_act_page_1_count', 'user_max_no_launch_days_hist', 'user_last_act_date', 'user_lastweek_act_2_freq', 'user_lastweek_video_freq', 'user_lastweek_act_2_count', 'user_lastweek_launch_freq', 'user_max_no_activity_days_lastweek_hist_dist', 'user_lastweek_act_page_2_count', 'user_var_continue_activity_times_lastweek', 'user_kurt_continue_activity_days_hist', 'user_launch_range_percent', 'user_activity_div_launch_days_hist', 'user_hist_act_video_meancount', 'user_hist_video_activity_types', 'user_activity_range_percent', 'user_5daybefore_act_page_1_count', 'user_min_continue_activity_times_5daywin', 'user_lastweek_hist_act_3_count_dist', 'user_lastweek_hist_act_page_3_count_dist', 'user_mean_createvideo_date_lastweek_hist_dist', 'user_4daybefore_act_page_4_count', 'user_kurt_no_launch_days_6daywin', 'user_max_continue_createvideo_days_4daywin', 'user_2daybefore_act_1_count', 'user_max_continue_createvideo_days_hist', '(register_type-user_lastweek_hist_act_3_count_dist)', '(register_type+user_last_act_date)', '(register_type/user_last_act_date)', '(register_type-user_lastweek_launch)', '(register_type*user_mean_continue_activity_days_hist)', '(register_type-user_min_continue_launch_times_lastweek)', '(register_type/user_min_continue_launch_times_lastweek)', '(register_type*user_lastweek_launchday)', '(device_type*user_activity_div_launch_days_hist)', '(device_type/user_lastweek_launch)', '(user_reg_days*user_lastweek_act_2_freq)', '(user_reg_days+user_hist_act_freq)', '(user_lastweek_launchday-user_min_continue_launch_days_lastweek)', '(user_lastweek_launchday-user_min_continue_launch_times_lastweek)', '(user_last_launch_dist/user_lastweek_act_2_freq)', '(user_hist_launchday/user_lastweek_actcount)', '(user_hist_act_0_count-user_kurt_continue_activity_days_hist)', '(user_mean_continue_activity_days_hist-user_hist_video_activity_types)', '(user_lastweek_act*user_max_continue_createvideo_days_4daywin)', '(user_lastweek_act_page_3_count*user_activity_days_lastweek)', '(user_kurt_continue_activity_days_hist/user_hist_video_activity_types)', '(user_launch_range_percent*user_lastweek_act_2_freq)', '(user_min_continue_activity_times_5daywin*user_lastweek_launch)', '(user_lastweek_hist_act_3_count_dist-user_2daybefore_act_1_count)', '(user_max_continue_createvideo_days_4daywin*user_lastweek_act_2_count)', '(user_2daybefore_act_1_count*user_lastweek_hist_act_page_3_count_dist)', '(user_max_continue_createvideo_days_hist-user_hist_video_activity_types)']
len(cols)

83

In [5]:
df['(device_type-user_activity_days_lastweek)'] = df.device_type - df.user_activity_days_lastweek
df['(user_lastweek_launchday-user_min_continue_launch_days_lastweek)'] = df.user_lastweek_launchday - df.user_min_continue_launch_days_lastweek
df['(user_last_launch_dist/user_lastweek_act_video_uniquecount)'] = (df.user_last_launch_dist + 0.001) / (df.user_lastweek_act_video_uniquecount + 0.001)
df['(user_activity_days_hist-user_kurt_continue_activity_days_hist)'] = df.user_activity_days_hist - df.user_kurt_continue_activity_days_hist
df['(user_activity_days_hist/user_var_continue_activity_times_lastweek)'] = (df.user_activity_days_hist + 0.001) / (df.user_var_continue_activity_times_lastweek + 0.001)
df['(user_activity_days_lastweek-user_max_continue_launch_times_hist)'] = df.user_activity_days_lastweek - df.user_max_continue_launch_times_hist
df['(user_activity_days_lastweek/user_max_continue_activity_days_lastweek)'] = (df.user_activity_days_lastweek + 0.001) / (df.user_max_continue_activity_days_lastweek + 0.001)
df['(user_lastweek_actcount-user_activity_days_lastweek)'] = df.user_lastweek_actcount - df.user_activity_days_lastweek
df['(user_lastweek_act_video_uniquecount-user_max_continue_activity_days_lastweek)'] = df.user_lastweek_act_video_uniquecount - df.user_max_continue_activity_days_lastweek
df['(user_min_continue_launch_days_lastweek*user_2daybefore_act_1_count)'] = df.user_min_continue_launch_days_lastweek * df.user_2daybefore_act_1_count
df['(user_hist_act_author_count*user_max_continue_createvideo_days_4daywin)'] = df.user_hist_act_author_count * df.user_max_continue_createvideo_days_4daywin
df['(user_max_launch_daytimes_lastweek*user_5daybefore_act_page_1_count)'] = df.user_max_launch_daytimes_lastweek * df.user_5daybefore_act_page_1_count
df['(user_max_launch_daytimes_lastweek*user_lastweek_act_page_3_count)'] = df.user_max_launch_daytimes_lastweek * df.user_lastweek_act_page_3_count

In [6]:
df['(register_type+user_lastweek_video_freq)'] = df.register_type + df.user_lastweek_video_freq
df['(register_type*user_hist_launchday)'] = df.register_type * df.user_hist_launchday
df['(user_hist_launchday/user_2daybefore_act_1_count)'] = (df.user_hist_launchday + 0.001) / (df.user_2daybefore_act_1_count + 0.001)
df['(user_activity_days_lastweek-user_mean_continue_activity_days_lastweek)'] = df.user_activity_days_lastweek - df.user_mean_continue_activity_days_lastweek
df['(user_min_continue_launch_times_lastweek*user_lastweek_hist_act_3_count_dist)'] = df.user_min_continue_launch_times_lastweek * df.user_lastweek_hist_act_3_count_dist
df['(user_mean_continue_activity_days_lastweek+user_max_no_activity_days_lastweek_hist_dist)'] = df.user_mean_continue_activity_days_lastweek + df.user_max_no_activity_days_lastweek_hist_dist
df['(user_lastweek_act_0_freq-user_4daybefore_act_page_4_count)'] = df.user_lastweek_act_0_freq - df.user_4daybefore_act_page_4_count
df['(user_max_continue_launch_days_hist*user_5daybefore_act_page_1_count)'] = df.user_max_continue_launch_days_hist - df.user_5daybefore_act_page_1_count
df['(user_var_continue_activity_times_lastweek*user_5daybefore_act_page_1_count)'] = df.user_var_continue_activity_times_lastweek * df.user_5daybefore_act_page_1_count
df['(user_launch_range_percent*user_4daybefore_act_page_4_count)'] = df.user_launch_range_percent * df.user_4daybefore_act_page_4_count
df['(user_launch_range_percent/user_4daybefore_act_page_4_count)'] = (df.user_launch_range_percent + 0.001) / (df.user_4daybefore_act_page_4_count + 0.001)
df['(user_5daybefore_act_page_1_count/user_lastweek_launch)'] = (df.user_5daybefore_act_page_1_count + 0.001) / (df.user_lastweek_launch + 0.001)
df['(user_5daybefore_act_page_1_count*user_min_continue_launch_days_lastweek)'] = df.user_5daybefore_act_page_1_count * df.user_min_continue_launch_days_lastweek
df['(user_4daybefore_act_page_4_count/user_max_launch_daytimes_lastweek)'] = (df.user_4daybefore_act_page_4_count + 0.001) / (df.user_max_launch_daytimes_lastweek + 0.001)

In [102]:
df['(register_type-user_lastweek_hist_act_3_count_dist)'] = df.register_type - df.user_lastweek_hist_act_3_count_dist
df['(register_type+user_last_act_date)'] = df.register_type + df.user_last_act_date
df['(register_type/user_last_act_date)'] = (df.register_type+0.001) / (df.user_last_act_date+0.001)
df['(register_type-user_lastweek_launch)'] = df.register_type - df.user_lastweek_launch
df['(register_type*user_mean_continue_activity_days_hist)'] = df.register_type * df.user_mean_continue_activity_days_hist
df['(register_type-user_min_continue_launch_times_lastweek)'] = df.register_type - df.user_min_continue_launch_times_lastweek
df['(register_type/user_min_continue_launch_times_lastweek)'] = (df.register_type+0.001) / (df.user_min_continue_launch_times_lastweek+0.001)
df['(register_type*user_lastweek_launchday)'] = df.register_type * df.user_lastweek_launchday
df['(device_type*user_activity_div_launch_days_hist)'] = df.device_type * df.user_activity_div_launch_days_hist
df['(device_type/user_lastweek_launch)']  = (df.device_type+0.001) / (df.user_lastweek_launch+0.001)
df['(user_reg_days*user_lastweek_act_2_freq)'] = df.user_reg_days * df.user_lastweek_act_2_freq
df['(user_reg_days+user_hist_act_freq)'] = df.user_reg_days + df.user_hist_act_freq
df['(user_lastweek_launchday-user_min_continue_launch_days_lastweek)'] = df.user_lastweek_launchday - df.user_min_continue_launch_days_lastweek
df['(user_lastweek_launchday-user_min_continue_launch_times_lastweek)'] = df.user_lastweek_launchday - df.user_min_continue_launch_times_lastweek
df['(user_last_launch_dist/user_lastweek_act_2_freq)'] = (df.user_last_launch_dist+0.001) / (df.user_lastweek_act_2_freq+0.001)
df['(user_hist_launchday/user_lastweek_actcount)'] = (df.user_hist_launchday+0.001) / (df.user_lastweek_actcount+0.001)
df['(user_hist_act_0_count-user_kurt_continue_activity_days_hist)'] = df.user_hist_act_0_count - df.user_kurt_continue_activity_days_hist
df['(user_mean_continue_activity_days_hist-user_hist_video_activity_types)'] = df.user_mean_continue_activity_days_hist - df.user_hist_video_activity_types
df['(user_lastweek_act*user_max_continue_createvideo_days_4daywin)'] = df.user_lastweek_act * df.user_max_continue_createvideo_days_4daywin
df['(user_lastweek_act_page_3_count*user_activity_days_lastweek)'] = df.user_lastweek_act_page_3_count * df.user_activity_days_lastweek
df['(user_kurt_continue_activity_days_hist/user_hist_video_activity_types)'] = (df.user_kurt_continue_activity_days_hist+0.001) / (df.user_hist_video_activity_types+0.001)
df['(user_launch_range_percent*user_lastweek_act_2_freq)'] = df.user_launch_range_percent * df.user_lastweek_act_2_freq
df['(user_min_continue_activity_times_5daywin*user_lastweek_launch)'] = df.user_min_continue_activity_times_5daywin * df.user_lastweek_launch
df['(user_lastweek_hist_act_3_count_dist-user_2daybefore_act_1_count)'] = df.user_lastweek_hist_act_3_count_dist - df.user_2daybefore_act_1_count
df['(user_max_continue_createvideo_days_4daywin*user_lastweek_act_2_count)'] = df.user_max_continue_createvideo_days_4daywin * df.user_lastweek_act_2_count
df['(user_2daybefore_act_1_count*user_lastweek_hist_act_page_3_count_dist)'] = df.user_2daybefore_act_1_count * df.user_lastweek_hist_act_page_3_count_dist
df['(user_max_continue_createvideo_days_hist-user_hist_video_activity_types)'] = df.user_max_continue_createvideo_days_hist - df.user_hist_video_activity_types

## pre-selection

In [73]:
df_trend = pd.read_csv('./features/trend_df_b.csv')
df_trend = df_trend.drop(['user_id', 'data_weeknum', 'data_weekstart', 'data_weekend', 'label'], axis=1)

df_trend.shape
df_trend.head()

(124709, 7)

,trend_7,trend_6,trend_5,trend_4,trend_3,trend_2,trend_1
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,0.034483,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2,0.009615,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
3,0.563025,0.320000,0.389189,0.043478,0.057143,0.070423,0.083333
4,2.177778,0.413793,0.392157,0.556034,0.214559,0.385714,0.847458


In [74]:
cand_cols = []

In [75]:
df = pd.concat([df, df_trend], axis=1)
cand_cols = cand_cols + list(df_trend.columns)
df.shape
len(cols)

(198057, 102)

83

In [76]:
df_group = pd.read_csv('./features/set_group_noleak_b.csv')
df_group.shape

(51480, 2)

In [77]:
df = df.merge(df_group, on=['user_id'], how='left')
df.shape

(198057, 103)

In [78]:
df = pd.concat([df, pd.get_dummies(df.set_group, prefix='set_group')], axis=1)
cand_cols = cand_cols + list(pd.get_dummies(df.set_group, prefix='set_group').columns)
df = df.drop(['set_group'], axis=1)
df.shape
len(cols)

(198057, 106)

83

In [79]:
df_group = pd.read_csv('./features/group_noleak_b.csv')
df_group.shape

(51480, 2)

In [80]:
df = df.merge(df_group, on=['user_id'], how='left')
df.shape

(198057, 107)

In [81]:
df = pd.concat([df, pd.get_dummies(df.group, prefix='group')], axis=1)
cand_cols = cand_cols + list(pd.get_dummies(df.group, prefix='group').columns)
df = df.drop(['group'], axis=1)
df.shape

(198057, 121)

In [104]:
df.isnull().sum()

user_id                                                                       0
data_weeknum                                                                  0
data_weekstart                                                                0
label                                                                         0
register_type                                                                 0
device_type                                                                   0
user_reg_days                                                                 0
user_2daybefore_act_1_count                                                   0
user_4daybefore_act_page_4_count                                              0
user_5daybefore_act_page_1_count                                              0
user_activity_days_hist                                                       0
user_activity_days_lastweek                                                   0
user_hist_act_0_count                   

In [109]:
df.shape
df[df.launch_d_type_rate.isnull()].head()

(198057, 101)

,user_id,data_weeknum,data_weekstart,label,register_type,device_type,user_reg_days,user_2daybefore_act_1_count,user_4daybefore_act_page_4_count,user_5daybefore_act_page_1_count,user_activity_days_hist,user_activity_days_lastweek,user_hist_act_0_count,user_hist_act_author_count,user_hist_act_video_meancount,user_hist_actcount,user_hist_launchday,user_hist_video_activity_types,user_kurt_continue_activity_days_hist,user_kurt_no_launch_days_6daywin,user_last_act_date,user_last_launch_dist,user_lastweek_act,user_lastweek_act_0,user_lastweek_act_2_count,user_lastweek_act_page_1_count,user_lastweek_act_page_2_count,user_lastweek_act_page_3_count,user_lastweek_act_video_uniquecount,user_lastweek_actcount,user_lastweek_launch,user_lastweek_launchday,user_max_continue_activity_days_lastweek,user_max_continue_createvideo_days_4daywin,user_max_continue_createvideo_days_hist,user_max_continue_launch_days_hist,user_max_continue_launch_days_lastweek,user_max_continue_launch_times_hist,user_max_continue_launch_times_lastweek,user_max_launch_daytimes_lastweek,user_max_no_launch_days_hist,user_mean_continue_activity_days_hist,user_mean_continue_activity_days_lastweek,user_mean_continue_activity_times_hist,user_mean_continue_launch_days_hist,user_mean_continue_launch_days_lastweek,user_mean_continue_launch_times_hist,user_mean_continue_launch_times_lastweek,user_mean_no_launch_days_hist,user_min_activity_daytimes_lastweek,user_min_continue_activity_days_lastweek,user_min_continue_activity_times_5daywin,user_min_continue_launch_days_lastweek,user_min_continue_launch_times_lastweek,user_var_continue_activity_times_lastweek,user_hist_launch_freq,user_lastweek_launch_freq,user_lastweek_video_freq,user_hist_act_freq,user_lastweek_act_0_freq,user_lastweek_act_2_freq,user_activity_div_launch_days_hist,user_lastweek_hist_act_3_count_dist,user_lastweek_hist_act_page_3_count_dist,user_max_no_activity_days_lastweek_hist_dist,user_mean_createvideo_date_lastweek_hist_dist,user_launch_range_percent,user_activity_range_percent,(register_type-user_lastweek_hist_act_3_count_dist),(register_type+user_last_act_date),(register_type/user_last_act_date),(register_type-user_lastweek_launch),(register_type*user_mean_continue_activity_days_hist),(register_type-user_min_continue_launch_times_lastweek),(register_type/user_min_continue_launch_times_lastweek),(register_type*user_lastweek_launchday),(device_type*user_activity_div_launch_days_hist),(device_type/user_lastweek_launch),(user_reg_days*user_lastweek_act_2_freq),(user_reg_days+user_hist_act_freq),(user_lastweek_launchday-user_min_continue_launch_days_lastweek),(user_lastweek_launchday-user_min_continue_launch_times_lastweek),(user_last_launch_dist/user_lastweek_act_2_freq),(user_hist_launchday/user_lastweek_actcount),(user_hist_act_0_count-user_kurt_continue_activity_days_hist),(user_mean_continue_activity_days_hist-user_hist_video_activity_types),(user_lastweek_act*user_max_continue_createvideo_days_4daywin),(user_lastweek_act_page_3_count*user_activity_days_lastweek),(user_kurt_continue_activity_days_hist/user_hist_video_activity_types),(user_launch_range_percent*user_lastweek_act_2_freq),(user_min_continue_activity_times_5daywin*user_lastweek_launch),(user_lastweek_hist_act_3_count_dist-user_2daybefore_act_1_count),(user_max_continue_createvideo_days_4daywin*user_lastweek_act_2_count),(user_2daybefore_act_1_count*user_lastweek_hist_act_page_3_count_dist),(user_max_continue_createvideo_days_hist-user_hist_video_activity_types),launch_r_type_rate,launch_d_type_rate,video_r_type_rate,video_d_type_rate,device_r_type_rate,device_d_type_rate
73348,167777,0,10,0,4,270,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,1,0,0.0,0.0000,-1.0,9,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,1,0.0,-1,0.0,0.0,-1,0.0,0.0,0.0,1.0,0.0,-1.0,0.0,-1.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.111111,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,4.0,3.0,-4.005005,4,0.0,4.0,4001.000000,0.0,0.0,270001.000000,0.0,9.000000,0.0,0

In [103]:
new_fea = pd.read_csv('./features/b/type_rate.csv')
new_fea['user_id'] = new_fea['user_id'].apply(lambda x:str(x)+'_a')
new_fea = pd.concat([new_fea, pd.read_csv('./features/b/type_rate_b.csv')])
print(new_fea.shape)
cand_cols = cand_cols + ['launch_r_type_rate','launch_d_type_rate','video_r_type_rate','video_d_type_rate','device_r_type_rate','device_d_type_rate']
df = df.merge(new_fea, on=['user_id', 'data_weekstart'], how='left')
assert(df.isnull().sum().sum() == 0)
print(df.shape)

(249766, 8)


AssertionError: 

In [18]:
df.to_csv('./features/features_zero.csv', index=None)

In [ ]:
df.shape
df.head()

In [112]:
df['user_id'] = df.user_id.astype(str)

train = df[(df.data_weeknum < df.data_weeknum.max()-1)]
val = df[(df.data_weeknum == df.data_weeknum.max()-1) & (df.user_id.map(lambda x:'_a' not in x))]
test = df[df.data_weeknum == df.data_weeknum.max()]
trainval = df[df.data_weeknum <df.data_weeknum.max()]

## XGB feature selection

In [30]:
len(cols)
# len(cols2)
# len(cand_cols)

83

In [31]:
def evalerror(preds, dtrain):
    labels = dtrain.get_label()
    f1s = []
    for limit in np.arange(0.4, 0.44, 0.01):
        pred = [int(i>limit) for i in preds]
        f1s.append(f1_score(labels, pred))
    
    
#     pred = [1. if i>0.45 else 0. for i in preds]
    # return a pair metric_name, result
    # since preds are margin(before logistic transformation, cutoff at 0)
    return 'f1', -1*max(f1s) #-1*f1_score(labels, pred)

In [110]:
# df = df.fillna(0)

In [114]:
params = {
        'objective': 'binary:logistic',
        'eta': 0.05,
#         'colsample_bytree': 0.3,
#         'subsample':0.9,
        'min_child_weight': 17,
        'max_depth': 5,
        'gamma':0.1,
        'seed': 2018,
        'missing': -1,
        'n_jobs':-1,
        'eval_metric':'auc',
#         'tree_method':'gpu_hist',
#         'max_bin':64,
#         'gpu_device':0
}
# f1 0.808476
len(cols)
dtrain = xgb.DMatrix(train[cols], label=train[['label']])
dval = xgb.DMatrix(val[cols], label=val[['label']])
model = xgb.train(params, dtrain, num_boost_round=20000, early_stopping_rounds=50, evals=[(dtrain, 'train'),(dval, 'val')],feval=evalerror,verbose_eval=100)

83

[0]	train-auc:0.867672	val-auc:0.881738	train-f1:-0.694072	val-f1:-0.656362
Multiple eval metrics have been passed: 'val-f1' will be used for early stopping.

Will train until val-f1 hasn't improved in 50 rounds.
[100]	train-auc:0.881089	val-auc:0.889987	train-f1:-0.813392	val-f1:-0.81104
Stopping. Best iteration:
[143]	train-auc:0.882654	val-auc:0.889807	train-f1:-0.814377	val-f1:-0.812283



In [ ]:
usecols:[132]	train-auc:0.886595	val-auc:0.888315	train-f1:-0.816726	val-f1:-0.807882
cols(after feature selection):[108]	train-auc:0.884968	val-auc:0.888525	train-f1:-0.815171	val-f1:-0.808063
cols(after zero):[106]	train-auc:0.885581	val-auc:0.888294	train-f1:-0.815903	val-f1:-0.807834
                    
cols(after group2 zero):

In [61]:
# feature_performance = {'set_group_0':0.805885,'set_group_1':0.806893,'set_group_2':0.806073,'trend_1':0.807128,'trend_2':0.807431,'trend_3':0.807076,'trend_4':0.807552,'trend_5':0.807503,'trend_6':0.80743,'trend_7':0.807243,'trend_act_0_1':0.807051,'trend_act_0_2':0.807225,'trend_act_0_3':0.808034,'trend_act_0_4':0.807852,'trend_act_0_5':0.807707,'trend_act_0_6':0.807914,'trend_act_0_7':0.806928,'trend_act_1_1':0.807652,'trend_act_1_2':0.807396,'trend_act_1_3':0.807496,'trend_act_1_4':0.807734,'trend_act_1_5':0.808187,'trend_act_1_6':0.807707,'trend_act_1_7':0.807724,'trend_act_2_1':0.807667,'trend_act_2_2':0.807745,'trend_act_2_3':0.807896,'trend_act_2_4':0.807481,'trend_act_2_5':0.807648,'trend_act_2_6':0.807811,'trend_act_2_7':0.807993,'trend_act_3_1':0.807671,'trend_act_3_2':0.807298,'trend_act_3_3':0.807583,'trend_act_3_4':0.807622,'trend_act_3_5':0.80759,'trend_act_3_6':0.807769,'trend_act_3_7':0.807606,'trend_act_4_1':0.807825,'trend_act_4_2':0.807825,'trend_act_4_3':0.807825,'trend_act_4_4':0.807825,'trend_act_4_5':0.807825,'trend_act_4_6':0.807825,'trend_act_4_7':0.807825,'trend_act_5_1':0.807825,'trend_act_5_2':0.807825,'trend_act_5_3':0.807825,'trend_act_5_4':0.807825,'trend_act_5_5':0.807825,'trend_act_5_6':0.807825,'trend_act_5_7':0.807825,'trend_page_0_1':0.807789,'trend_page_0_2':0.807067,'trend_page_0_3':0.806947,'trend_page_0_4':0.80757,'trend_page_0_5':0.807045,'trend_page_0_6':0.807595,'trend_page_0_7':0.807646,'trend_page_1_1':0.807512,'trend_page_1_2':0.807509,'trend_page_1_3':0.807487,'trend_page_1_4':0.80765,'trend_page_1_5':0.807631,'trend_page_1_6':0.807337,'trend_page_1_7':0.8073,'trend_page_2_1':0.806778,'trend_page_2_2':0.807309,'trend_page_2_3':0.807297,'trend_page_2_4':0.807183,'trend_page_2_5':0.807752,'trend_page_2_6':0.807502,'trend_page_2_7':0.807527,'trend_page_3_1':0.807844,'trend_page_3_2':0.807319,'trend_page_3_3':0.807231,'trend_page_3_4':0.807522,'trend_page_3_5':0.807649,'trend_page_3_6':0.807407,'trend_page_3_7':0.80737,'trend_page_4_1':0.80734,'trend_page_4_2':0.8076,'trend_page_4_3':0.807466,'trend_page_4_4':0.807888,'trend_page_4_5':0.807263,'trend_page_4_6':0.807843,'trend_page_4_7':0.807588}
feature_performance = {}
for i in tqdm_notebook(cand_cols):
    tmp_cols = cols.copy()
    tmp_cols.append(i)
    assert(len(tmp_cols) == 84)
    dtrain = xgb.DMatrix(train[tmp_cols], label=train[['label']])
    dval = xgb.DMatrix(val[tmp_cols], label=val[['label']])
    model = xgb.train(params, dtrain, num_boost_round=100, early_stopping_rounds=50, evals=[(dtrain, 'train'),(dval, 'val')],feval=evalerror,verbose_eval=0)
    feature_performance[i] = -model.best_score

In [62]:
feature_performance

{'trend_1': 0.811019,
 'trend_2': 0.811266,
 'trend_3': 0.811015,
 'trend_4': 0.81135,
 'trend_5': 0.81177,
 'trend_6': 0.810787,
 'trend_7': 0.811013}

In [23]:
feature_performance

{'group_0': 0.807744,
 'group_1': 0.817314,
 'group_10': 0.80763,
 'group_11': 0.806826,
 'group_12': 0.807472,
 'group_13': 0.808312,
 'group_14': 0.808102,
 'group_2': 0.807219,
 'group_3': 0.807825,
 'group_4': 0.807825,
 'group_5': 0.807513,
 'group_6': 0.807649,
 'group_7': 0.807478,
 'group_8': 0.807597,
 'group_9': 0.812484,
 'set_group_0': 0.805885,
 'set_group_1': 0.806893,
 'set_group_2': 0.806073,
 'trend_1': 0.807128,
 'trend_2': 0.807431,
 'trend_3': 0.807076,
 'trend_4': 0.807552,
 'trend_5': 0.807503,
 'trend_6': 0.80743,
 'trend_7': 0.807243,
 'trend_act_0_1': 0.807051,
 'trend_act_0_2': 0.807225,
 'trend_act_0_3': 0.808034,
 'trend_act_0_4': 0.807852,
 'trend_act_0_5': 0.807707,
 'trend_act_0_6': 0.807914,
 'trend_act_0_7': 0.806928,
 'trend_act_1_1': 0.807652,
 'trend_act_1_2': 0.807396,
 'trend_act_1_3': 0.807496,
 'trend_act_1_4': 0.807734,
 'trend_act_1_5': 0.808187,
 'trend_act_1_6': 0.807707,
 'trend_act_1_7': 0.807724,
 'trend_act_2_1': 0.807667,
 'trend_act_2_2

In [31]:
feature_performance

{'group_0': 0.808064,
 'group_1': 0.805575,
 'group_10': 0.807257,
 'group_11': 0.807622,
 'group_12': 0.807591,
 'group_13': 0.807648,
 'group_14': 0.806443,
 'group_2': 0.807724,
 'group_3': 0.806751,
 'group_4': 0.807825,
 'group_5': 0.807825,
 'group_6': 0.807569,
 'group_7': 0.80722,
 'group_8': 0.807347,
 'group_9': 0.807825,
 'set_group_0': 0.805885,
 'set_group_1': 0.806893,
 'set_group_2': 0.806073,
 'trend_1': 0.807128,
 'trend_2': 0.807431,
 'trend_3': 0.807076,
 'trend_4': 0.807552,
 'trend_5': 0.807503,
 'trend_6': 0.80743,
 'trend_7': 0.807243,
 'trend_act_0_1': 0.807051,
 'trend_act_0_2': 0.807225,
 'trend_act_0_3': 0.808034,
 'trend_act_0_4': 0.807852,
 'trend_act_0_5': 0.807707,
 'trend_act_0_6': 0.807914,
 'trend_act_0_7': 0.806928,
 'trend_act_1_1': 0.807652,
 'trend_act_1_2': 0.807396,
 'trend_act_1_3': 0.807496,
 'trend_act_1_4': 0.807734,
 'trend_act_1_5': 0.808187,
 'trend_act_1_6': 0.807707,
 'trend_act_1_7': 0.807724,
 'trend_act_2_1': 0.807667,
 'trend_act_2_2

In [22]:
baseline = 0.808
good_features = []

for i in feature_performance.items():
    if i[1] > baseline:
        good_features.append(i[0])
len(good_features)

6

In [32]:
sorted(feature_performance.items(), key=lambda x:x[1], reverse=True)

[('trend_act_1_5', 0.808187),
 ('group_0', 0.808064),
 ('trend_act_0_3', 0.808034),
 ('trend_act_2_7', 0.807993),
 ('trend_act_0_6', 0.807914),
 ('trend_act_2_3', 0.807896),
 ('trend_page_4_4', 0.807888),
 ('trend_act_0_4', 0.807852),
 ('trend_page_3_1', 0.807844),
 ('trend_page_4_6', 0.807843),
 ('trend_act_4_5', 0.807825),
 ('trend_act_5_3', 0.807825),
 ('trend_act_5_7', 0.807825),
 ('trend_act_4_3', 0.807825),
 ('trend_act_5_4', 0.807825),
 ('trend_act_4_2', 0.807825),
 ('trend_act_5_2', 0.807825),
 ('trend_act_5_5', 0.807825),
 ('trend_act_4_1', 0.807825),
 ('group_4', 0.807825),
 ('trend_act_4_6', 0.807825),
 ('group_9', 0.807825),
 ('trend_act_5_1', 0.807825),
 ('trend_act_4_7', 0.807825),
 ('trend_act_5_6', 0.807825),
 ('trend_act_4_4', 0.807825),
 ('group_5', 0.807825),
 ('trend_act_2_6', 0.807811),
 ('trend_page_0_1', 0.807789),
 ('trend_act_3_6', 0.807769),
 ('trend_page_2_5', 0.807752),
 ('trend_act_2_2', 0.807745),
 ('trend_act_1_4', 0.807734),
 ('group_2', 0.807724),
 ('tr

## submit

In [53]:
# cand_cols = list(set(cand_cols) - set(['set_group_0', 'set_group_1', 'set_group_2']))
cand_cols = cand_cols + ['set_group_0', 'set_group_1', 'set_group_2']

In [33]:
features = [i[0] for i in sorted(feature_performance.items(), key=lambda x:x[1], reverse=True)]
features

['trend_act_1_5',
 'group_0',
 'trend_act_0_3',
 'trend_act_2_7',
 'trend_act_0_6',
 'trend_act_2_3',
 'trend_page_4_4',
 'trend_act_0_4',
 'trend_page_3_1',
 'trend_page_4_6',
 'trend_act_4_5',
 'trend_act_5_3',
 'trend_act_5_7',
 'trend_act_4_3',
 'trend_act_5_4',
 'trend_act_4_2',
 'trend_act_5_2',
 'trend_act_5_5',
 'trend_act_4_1',
 'group_4',
 'trend_act_4_6',
 'group_9',
 'trend_act_5_1',
 'trend_act_4_7',
 'trend_act_5_6',
 'trend_act_4_4',
 'group_5',
 'trend_act_2_6',
 'trend_page_0_1',
 'trend_act_3_6',
 'trend_page_2_5',
 'trend_act_2_2',
 'trend_act_1_4',
 'group_2',
 'trend_act_1_7',
 'trend_act_1_6',
 'trend_act_0_5',
 'trend_act_3_1',
 'trend_act_2_1',
 'trend_act_1_1',
 'trend_page_1_4',
 'trend_page_3_5',
 'trend_act_2_5',
 'group_13',
 'trend_page_0_7',
 'trend_page_1_5',
 'group_11',
 'trend_act_3_4',
 'trend_act_3_7',
 'trend_page_4_2',
 'trend_page_0_6',
 'group_12',
 'trend_act_3_5',
 'trend_page_4_7',
 'trend_act_3_3',
 'trend_page_0_4',
 'group_6',
 'trend_4',


In [53]:
for i in features:
    if 'group' in i:
        features.remove(i)

In [73]:
params = {
        'objective': 'binary:logistic',
        'eta': 0.05,
#         'colsample_bytree': 0.3,
#         'subsample':0.9,
        'min_child_weight': 17,
        'max_depth': 5,
        'gamma':0.1,
        'seed': 2018,
        'missing': -1,
        'n_jobs':10,
        'eval_metric':'auc',
#         'tree_method':'gpu_hist',
#         'max_bin':64,
#         'gpu_device':0
}
# f1 0.808476
len(cols2 + good_features)
dtrain = xgb.DMatrix(train[cols2 + good_features], label=train[['label']])
dval = xgb.DMatrix(val[cols2 + good_features], label=val[['label']])
model = xgb.train(params, dtrain, num_boost_round=20000, early_stopping_rounds=50, evals=[(dtrain, 'train'),(dval, 'val')],feval=evalerror,verbose_eval=100)

81

[0]	train-auc:0.870193	val-auc:0.877232	train-f1:-0.690067	val-f1:-0.654691
Multiple eval metrics have been passed: 'val-f1' will be used for early stopping.

Will train until val-f1 hasn't improved in 50 rounds.
[100]	train-auc:0.885321	val-auc:0.888298	train-f1:-0.814881	val-f1:-0.807468
[200]	train-auc:0.890285	val-auc:0.88829	train-f1:-0.8183	val-f1:-0.808331
Stopping. Best iteration:
[177]	train-auc:0.889376	val-auc:0.888405	train-f1:-0.817919	val-f1:-0.808844



In [64]:
features

['trend_act_1_5',
 'trend_act_0_3',
 'trend_act_2_7',
 'trend_act_0_6',
 'trend_act_2_3',
 'trend_page_4_4',
 'trend_act_0_4',
 'trend_page_3_1',
 'trend_page_4_6',
 'trend_act_4_5',
 'trend_act_5_3',
 'trend_act_5_7',
 'trend_act_4_3',
 'trend_act_5_4',
 'trend_act_4_2',
 'trend_act_5_2',
 'trend_act_5_5',
 'trend_act_4_1',
 'trend_act_4_6',
 'trend_act_5_1',
 'trend_act_4_7',
 'trend_act_5_6',
 'trend_act_4_4',
 'trend_act_2_6',
 'trend_page_0_1',
 'trend_act_3_6',
 'trend_page_2_5',
 'trend_act_2_2',
 'trend_act_1_4',
 'trend_act_1_7',
 'trend_act_1_6',
 'trend_act_0_5',
 'trend_act_3_1',
 'trend_act_2_1',
 'trend_act_1_1',
 'trend_page_1_4',
 'trend_page_3_5',
 'trend_act_2_5',
 'trend_page_0_7',
 'trend_page_1_5',
 'trend_act_3_4',
 'trend_act_3_7',
 'trend_page_4_2',
 'trend_page_0_6',
 'trend_act_3_5',
 'trend_page_4_7',
 'trend_act_3_3',
 'trend_page_0_4',
 'trend_4',
 'trend_page_2_7',
 'trend_page_3_4',
 'trend_page_1_1',
 'trend_page_1_2',
 'trend_5',
 'trend_page_2_6',
 'tr

In [65]:
# greedy search
tmp_features = cols2.copy()
good_features = []
best_f1 = 0
for i in tqdm_notebook(features):
    tmp_features.append(i)
    dtrain = xgb.DMatrix(train[tmp_features], label=train[['label']])
    dval = xgb.DMatrix(val[tmp_features], label=val[['label']])
    model = xgb.train(params, dtrain, num_boost_round=20000, early_stopping_rounds=50, evals=[(dtrain, 'train'),(dval, 'val')],feval=evalerror,verbose_eval=0)
    if -model.best_score > best_f1:
        good_features.append(i)
        best_f1 = -model.best_score
    else:
        tmp_features.remove(i)

In [108]:
# cols
good_features
best_f1
# ['trend_act_1_5', 'trend_act_2_7', 'trend_act_0_6', 'trend_act_2_1']
# 0.808506

['trend_act_1_5', 'trend_act_2_7', 'trend_act_0_6', 'trend_act_2_1']

0.808506

In [35]:
# cols2
good_features
best_f1
# ['trend_act_1_5','group_0','trend_act_0_3','trend_act_0_6','trend_act_3_6','trend_act_2_2','trend_act_3_5']
# 0.809263

['trend_act_1_5',
 'group_0',
 'trend_act_0_3',
 'trend_act_0_6',
 'trend_act_3_6',
 'trend_act_2_2',
 'trend_act_3_5']

0.809263

In [66]:
# cols2 no leak
good_features
best_f1

['trend_act_1_5',
 'trend_act_1_4',
 'trend_act_2_1',
 'trend_act_3_5',
 'trend_6',
 'trend_page_4_5',
 'trend_page_2_1']

0.808844

In [ ]:
1:[82]	train-auc:0.883361	val-auc:0.887955	train-f1:-0.815031	val-f1:-0.808187
2:[65]	train-auc:0.882083	val-auc:0.887499	train-f1:-0.814102	val-f1:-0.807672
9:[82]	train-auc:0.88356	val-auc:0.88779	train-f1:-0.814585	val-f1:-0.807847
                    
usercols:[132]	train-auc:0.886595	val-auc:0.888315	train-f1:-0.816726	val-f1:-0.807882
cols2:[139]	train-auc:0.88737	val-auc:0.88847	train-f1:-0.816563	val-f1:-0.808156

In [39]:
sorted(model.get_score().items(), key=lambda x:x[1], reverse=True)

[('device_type', 405),
 ('user_hist_act_video_meancount', 199),
 ('trend_act_0_3', 180),
 ('user_hist_launch_freq', 167),
 ('user_last_launch_dist', 154),
 ('user_hist_act_freq', 151),
 ('user_launch_range_percent', 148),
 ('(register_type+user_lastweek_video_freq)', 148),
 ('user_hist_act_author_count', 135),
 ('user_mean_continue_activity_times_hist', 132),
 ('user_lastweek_act_page_1_count', 130),
 ('user_activity_range_percent', 129),
 ('trend_act_0_6', 112),
 ('trend_act_1_5', 102),
 ('user_last_act_date', 99),
 ('user_hist_actcount', 98),
 ('user_min_continue_activity_times_5daywin', 82),
 ('user_lastweek_act_0_freq', 73),
 ('user_lastweek_act_video_uniquecount', 66),
 ('user_lastweek_act_page_3_count', 63),
 ('user_lastweek_actcount', 62),
 ('user_min_activity_daytimes_lastweek', 55),
 ('user_lastweek_act_page_2_count', 54),
 ('user_lastweek_hist_act_page_3_count_dist', 54),
 ('(user_lastweek_act_0_freq-user_4daybefore_act_page_4_count)', 54),
 ('register_type', 53),
 ('user_las

In [33]:
pred = model.predict(dval, ntree_limit=model.best_ntree_limit)
limits = []
for limit in np.arange(0.4, 0.46, 0.01):
    preds = [int(i>limit) for i in pred]
    score = f1_score(dval.get_label(), preds)
    limits.append((limit, -1*score))
    print(limit, score, precision_score(dval.get_label(), preds), recall_score(dval.get_label(), preds))#0.807701253391911 0.763335287221571 0.8575426658618229
limits.sort(key=lambda x:x[1])
best_limit = round(limits[0][0],2)
print('best_limit:', best_limit, 'score:', limits[0][1])

0.4 0.8106663899769136 0.7692610643430315 0.8567825419453887
0.41000000000000003 0.8112676056338027 0.7736543627499751 0.8527250795043316
0.42000000000000004 0.811016393442623 0.7773922663046211 0.847680666739774
0.43000000000000005 0.8122825513969425 0.7821033397624607 0.8448843074898563
0.44000000000000006 0.8117840475180315 0.7860223888261272 0.8392915889900209
0.45000000000000007 0.8117286418765828 0.7898531929242102 0.8348503125342691
best_limit: 0.43 score: -0.8122825513969425


In [116]:
len(cols)
dtrainval = xgb.DMatrix(trainval[cols], label=trainval[['label']])
dtest = xgb.DMatrix(test[cols], label=test[['label']])
model.best_iteration
model = xgb.train(params, dtrainval, num_boost_round=int(model.best_iteration*1.1))

83

143

In [118]:
test['pred'] = model.predict(dtest)
save = test[test.pred>0.41].sort_values('pred', ascending=False)[['user_id']]
print(save.shape)

(23978, 1)


/usr/local/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [121]:
test[['user_id', 'pred']].to_csv('./result/b/xgb_zero.csv',index=False, header=None)

In [42]:
save.to_csv('./result/b/xgb_zero_0629_limit042.csv',index=False, header=None)